In [2]:
import os
import time
import json
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings as CommunityHuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chains import LLMChain

c:\Users\Amanda\anaconda3\envs\guia_inteligente_viagem\lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [3]:
load_dotenv()

# Configura o LLM da Groq e o modelo de embeddings do Hugging Face
llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0.7)
llm_router = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0)
embeddings = CommunityHuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Conecta-se ao Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "travel-assistant-index"
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

# Função auxiliar para formatar os documentos de contexto
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

C:\Users\Amanda\AppData\Local\Temp\ipykernel_7768\3810665054.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = CommunityHuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
itinerary_prompt = ChatPromptTemplate.from_template("""
    Você é um assistente de viagens especializado em criar roteiros detalhados.
    Use o contexto abaixo para gerar um roteiro de viagem personalizado para o usuário.
    Contexto: {context}
    Pergunta do usuário: {question}
    Responda em formato de roteiro com dias, horários e sugestões de atividades.
    Roteiro:""")

logistics_prompt = ChatPromptTemplate.from_template("""
    Você é um assistente de viagens focado em logística e transporte.
    Responda à pergunta do usuário de forma direta, prática e objetiva.
    Pergunta do usuário: {question}
    Resposta:""")

local_info_prompt = ChatPromptTemplate.from_template("""
    Você é um especialista em informações locais e turismo.
    Use o contexto abaixo para responder à pergunta do usuário de forma clara e objetiva.
    Contexto: {context}
    Pergunta do usuário: {question}
    Responda de forma direta e concisa. Se a informação não estiver disponível, diga que não sabe.
    Informação:""")

translation_prompt = ChatPromptTemplate.from_template("""
    Você é um assistente de tradução e guia de frases úteis para viagens.
    Traduza a frase do usuário.
    Exemplo: "Onde fica o banheiro?" -> "Where is the bathroom?"
    Frase do usuário: {question}
    Tradução/Frase:""")

# Cadeias 
itinerary_chain = ({
    "context": RunnableLambda(lambda x: vectorstore.as_retriever(search_kwargs={"filter": x.get("filtros", {})}).invoke(x["question"])) | RunnableLambda(format_docs),
    "question": RunnablePassthrough()
} | itinerary_prompt | llm | StrOutputParser())

logistics_chain = ({ "question": RunnablePassthrough() } | logistics_prompt | llm | StrOutputParser())

local_info_chain = ({
    "context": RunnableLambda(lambda x: vectorstore.as_retriever(search_kwargs={"filter": x.get("filtros", {})}).invoke(x["question"])) | RunnableLambda(format_docs),
    "question": RunnablePassthrough()
} | local_info_prompt | llm | StrOutputParser())

translation_chain = ({ "question": RunnablePassthrough() } | translation_prompt | llm | StrOutputParser())

# Prompt e Cadeia do Roteador 
router_template_simples = """
    Sua tarefa é classificar uma consulta de usuário em uma das seguintes especialidades.
    Você deve retornar APENAS O NOME DA ESPECIALIDADE, sem qualquer outra palavra, explicação ou pontuação.

    Especialidades:
    - roteiro_viagem
    - logistica_transporte
    - informacao_local
    - traducao_idiomas

    Consulta: {query}
    Especialidade:
"""
router_prompt_simples = ChatPromptTemplate.from_template(router_template_simples)
router_chain = LLMChain(llm=llm_router, prompt=router_prompt_simples, output_parser=StrOutputParser())


In [ ]:
class GuiaTurismoInteligente:
    def __init__(self):
        # Mapeia as cadeias para o roteador
        self.destination_chains = {
            "roteiro_viagem": itinerary_chain,
            "logistica_transporte": logistics_chain,
            "informacao_local": local_info_chain,
            "traducao_idiomas": translation_chain,
        }
        # Incializa a cadeia do roteador
        self.router_chain = router_chain
        
        self.stats = {
            'consultas_total': 0,
            'por_especialidade': {'roteiro_viagem': 0, 'logistica_transporte': 0, 'informacao_local': 0, 'traducao_idiomas': 0, 'desconhecido': 0},
            'tempo_medio': 0,
            'historico': []
        }

    def processar_consulta(self, pergunta: str) -> str:
        start_time = time.time()
        
        try:
            # 1. Roteamento: Roda o roteador para obter a especialidade
            especialidade = self.router_chain.run(query=pergunta).strip().lower()
            
            # 2. Lógica de Fallback (para tratar saídas inesperadas do LLM)
            if especialidade not in self.destination_chains:
                print(f"Assistente: Roteador retornou '{especialidade}'. Usando fallback.")
                self.stats['por_especialidade']['desconhecido'] += 1
                return "Desculpe, não consegui entender a sua solicitação. Por favor, tente novamente de forma diferente."

            # 3. Executar a cadeia correta
            chain = self.destination_chains[especialidade]
            
            # As cadeias de RAG precisam de um dicionário como input
            if especialidade in ["roteiro_viagem", "informacao_local"]:
                response = chain.invoke({"question": pergunta, "filtros": {}})
            else:
                response = chain.invoke({"question": pergunta})

            # 4. Atualizar as estatísticas
            tempo_processamento = time.time() - start_time
            self.stats['consultas_total'] += 1
            self.stats['por_especialidade'][especialidade] += 1
            self.stats['tempo_medio'] = (
                (self.stats['tempo_medio'] * (self.stats['consultas_total'] - 1) + tempo_processamento) / 
                self.stats['consultas_total']
            )
            
            print(f"Assistente: 🎯 Roteado para '{especialidade}'. Tempo: {tempo_processamento:.2f}s")
            
            return response
            
        except Exception as e:
            print(f"Assistente: Ocorreu um erro: {e}")
            return "Desculpe, ocorreu um erro. Por favor, tente novamente mais tarde."


In [8]:
# Teste de Roteiro de Viagem
guia = GuiaTurismoInteligente()
pergunta = "Crie um roteiro cultural de 3 dias em Paris."
resposta = guia.processar_consulta(pergunta)
print(f"---")
print(f"💬 Pergunta: {pergunta}")
print(f"🤖 Resposta: {resposta}")

Assistente: 🎯 Roteado para 'roteiro_viagem'. Tempo: 2.85s
---
💬 Pergunta: Crie um roteiro cultural de 3 dias em Paris.
🤖 Resposta: **Roteiro Cultural de 3 Dias em Paris**

**Dia 1: Arte e História**

* 9:00 - 10:00: Inicie o dia visitando o **Museu do Louvre** (Metro: Palais-Royal–Musée du Louvre). Gaste cerca de 2 horas explorando as obras-primas como a Mona Lisa e a Vênus de Milo.
* 10:30 - 12:30: Após visitar o Louvre, caminhe até a **Catedral de Notre-Dame** (Metro: Cité). Aproveite para observar a arquitetura gótica e o ambiente histórico.
* 13:00 - 14:30: Almoce no **Le Jules Verne** (Torre Eiffel), um restaurante gourmet localizado na Torre Eiffel. Desfrute de uma vista espetacular enquanto saborea a culinária francesa.
* 15:00 - 17:00: Visite o **Montmartre e a Basílica de Sacré-Cœur** (Metro: Anvers ou Blanche). Explore as ruas estreitas e artísticas do bairro, e suba até a basílica para desfrutar de uma vista incrível de Paris.

**Dia 2: Arquitetura e História**

* 9:30 - 11:

In [ ]:
# Teste de Informação Local
guia = GuiaTurismoInteligente()
pergunta = "Como chegar ao Coliseu?"
resposta = guia.processar_consulta(pergunta)
print(f"---")
print(f"💬 Pergunta: {pergunta}")
print(f"🤖 Resposta: {resposta}")

Assistente: 🎯 Roteado para 'informacao_local'. Tempo: 1.44s
---
💬 Pergunta: Como chegar ao Coliseu?
🤖 Resposta: Para chegar ao Coliseu, você pode utilizar o transporte público, como ônibus ou metrô. A estação de metro mais próxima é Colosseo, que fica a poucos minutos a pé do Coliseu. Além disso, você também pode caminhar a partir da Piazza Venezia ou utilizar táxi ou aplicativos de transporte.


In [ ]:
# Teste de Tradução
guia = GuiaTurismoInteligente()
pergunta = "Traduza a frase 'muito obrigado pela ajuda' para o italiano."
resposta = guia.processar_consulta(pergunta)
print(f"---")
print(f"💬 Pergunta: {pergunta}")
print(f"🤖 Resposta: {resposta}")

Assistente: 🎯 Roteado para 'traducao_idiomas'. Tempo: 1.30s
---
💬 Pergunta: Traduza a frase 'muito obrigado pela ajuda' para o italiano.
🤖 Resposta: Muito obrigado pela ajuda -> Grazie mille per l'aiuto.

Nota: 'Grazie mille' é uma expressão italiana que significa 'muito obrigado' ou 'muito obrigada', e 'per l'aiuto' significa 'pela ajuda'.


In [ ]:
# Teste de Logística
guia = GuiaTurismoInteligente()
pergunta = "Melhor forma de transporte em Tóquio?"
resposta = guia.processar_consulta(pergunta)
print(f"---")
print(f"💬 Pergunta: {pergunta}")
print(f"🤖 Resposta: {resposta}")

Assistente: 🎯 Roteado para 'logistica_transporte'. Tempo: 1.38s
---
💬 Pergunta: Melhor forma de transporte em Tóquio?
🤖 Resposta: Em Tóquio, a melhor forma de transporte depende do seu destino e preferências pessoais. No entanto, aqui estão algumas opções populares:

- **Metro**: O sistema de metrô em Tóquio é um dos mais eficientes do mundo. É uma opção rápida e econômica para se deslocar pela cidade.
- **Trens**: Os trens são outra opção popular em Tóquio. Eles cobrem uma grande área da cidade e são frequentes.
- **Ônibus**: Os ônibus são uma opção mais barata em comparação com o metrô ou trens, mas podem ser mais lentos.
- **Táxi**: Os táxis são uma opção conveniente, especialmente à noite, mas podem ser mais caros do que as outras opções.
- **Bicicleta**: Tóquio tem uma rede de bicicletas compartilhadas e uma lei que permite a circulação de bicicletas em algumas ruas.

Se você estiver procurando por uma opção rápida e conveniente, recomendo usar o metrô. Se você estiver com orçamen